In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../..")
print(os.getcwd())

import torch
from modules.amine_blend_pipeline import AmineBlendPipeline
import modules.datasplit_module as dsm
from modules.model_loader import load_model
from torch_geometric.loader import DataLoader
import numpy as np
import random

# --- Reproducibility settings ---
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 
seed = 21
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.use_deterministic_algorithms(False)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

In [ ]:
# --- Process data ---
pipeline = AmineBlendPipeline(components_csv='datasets/components.csv')
canonical_data, graph_list = pipeline.run_pipeline(raw_csv='datasets/dataset.csv')
pipeline.save_canonical_df(canonical_data, 'datasets/canonical_data.csv')
random.shuffle(graph_list)
train_raw, val_raw, test_raw, train_std, val_std, test_std, stats = \
    dsm.standardized_system_disjoint_split(graph_list, random_state=seed)

In [ ]:
# Load model
model, stats = load_model('notebooks/inference_phase/model_weights/best_model.pt', return_stats=True)
model = model.to('cuda')
# Load data
train_loader = DataLoader(
    dataset=train_std,
    batch_size=128,
    shuffle=True,
    follow_batch=['component_batch']
)

val_loader = DataLoader(
    dataset=val_std,
    batch_size=128,
    shuffle=False,
    follow_batch=['component_batch']
)

test_loader = DataLoader(
    dataset=test_std,
    batch_size=128,
    shuffle=False,
    follow_batch=['component_batch']
)

## GRAPH BLOCK ANALYSIS

In [ ]:
# --- Process data ---
pipeline = AmineBlendPipeline(components_csv='datasets/components.csv')
data_gamma, graph_gamma = pipeline.run_pipeline(raw_csv='datasets/data_gamma.csv')

In [ ]:
# Step 1: Standardize the graph_gamma data using existing stats
from copy import deepcopy

graph_gamma_std = []
for data in graph_gamma:
    data_std = deepcopy(data)
    data_std.T = (data.T - stats['T']['mean']) / stats['T']['std']
    data_std.T_mean = stats['T']['mean']
    data_std.T_std = stats['T']['std']
    graph_gamma_std.append(data_std)

print(f"Standardized {len(graph_gamma_std)} data points")
print(f"First sample: {graph_gamma_std[0].component_names}")
print(f"Mole fractions range: {graph_gamma_std[0].component_mole_frac} to {graph_gamma_std[-1].component_mole_frac}")

In [ ]:
# Step 1: Standardize temperature and extract latent vectors
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
model.eval()

x_1_list = []
gamma_1_list = []
gamma_h2o_list = []

with torch.no_grad():
    for data in graph_gamma:
        data_std = deepcopy(data)
        data_std.T = (data.T - stats['T']['mean']) / stats['T']['std']
        data_std.T_mean = stats['T']['mean']
        data_std.T_std = stats['T']['std']
        loader = DataLoader(
            dataset=[data_std],
            batch_size=1,
            shuffle=False,
            follow_batch=['component_batch']
        )
        batched_data = next(iter(loader)).to('cuda')
        
        # Extract latent vectors
        node_emb, comp_emb = model.graph_block(batched_data.x, batched_data.edge_index, 
                                                batched_data.edge_attr, batched_data.mol_batch)
        
        mixture_embs, latent_vectors, mole_frac = model.mixture_layer(
            comp_emb, 
            batched_data.component_mole_frac,
            batched_data.component_batch_batch,
            batched_data.T,
            batched_data.T_std,
            batched_data.T_mean,
            track_grad=False
        )
        
        # Calculate gamma_i
        gamma_i = latent_vectors.mean(dim=1).cpu().numpy()
        
        x_1 = batched_data.component_mole_frac[0].cpu().item()
        x_1_list.append(x_1)
        gamma_1_list.append(gamma_i[0])
        gamma_h2o_list.append(gamma_i[1])

print(f"Extracted {len(x_1_list)} points")
print(f"x_MDEA range: {min(x_1_list):.3f} to {max(x_1_list):.3f}")

In [ ]:
# Step 2: Plot activity coefficients
fig, ax = plt.subplots(figsize=(10, 8))

# Convert to arrays and sort by x_MEA
x_1_array = np.array(x_1_list)
gamma_1_array = np.array(gamma_1_list)
gamma_h2o_array = np.array(gamma_h2o_list)

sorted_idx = np.argsort(x_1_array)
x_1_sorted = x_1_array[sorted_idx]
gamma_1_sorted = gamma_1_array[sorted_idx]
gamma_h2o_sorted = gamma_h2o_array[sorted_idx]

# Plot
""" ax.plot(x_1_sorted, gamma_1_sorted, 'o-', linewidth=2.5, markersize=8, 
        label='γ_1', color='#1f77b4') """
ax.plot(x_1_sorted, gamma_h2o_sorted, 's-', linewidth=2.5, markersize=8, 
        label='γ_H2O', color='#2ca02c')

ax.set_xlabel('Mole fraction of Amine (x_1)', fontsize=14)
ax.set_ylabel('Activity coefficient (γᵢ)', fontsize=14)
ax.set_title('Activity Coefficients vs Composition\nMDEA + H2O at T=313 K', 
             fontsize=15, fontweight='bold')
ax.set_xscale('linear')
ax.grid(True, alpha=0.3, which='both')
ax.legend(fontsize=13)
plt.tight_layout()
plt.show()

print("Activity coefficient plot complete!")
print(f"γ_1 range: {gamma_1_sorted.min():.4f} to {gamma_1_sorted.max():.4f}")
print(f"γ_H2O range: {gamma_h2o_sorted.min():.4f} to {gamma_h2o_sorted.max():.4f}")

In [ ]:
# Step 3: Calculate excess Gibbs energy
R = 8.314  # J/(mol·K)
T = 313.15  # K

# Calculate G_excess for each component
G_excess_1 = gamma_1_sorted * R * T
G_excess_h2o = gamma_h2o_sorted * R * T

# Calculate molar excess Gibbs energy of mixing
x_h2o_sorted = 1 - x_1_sorted  # x_H2O = 1 - x_1
G_excess_mix = R * T * (x_1_sorted * np.log(gamma_1_sorted) + x_h2o_sorted * np.log(gamma_h2o_sorted))

print(f"G_excess_1 range: {G_excess_1.min():.2f} to {G_excess_1.max():.2f} J/mol")
print(f"G_excess_H2O range: {G_excess_h2o.min():.2f} to {G_excess_h2o.max():.2f} J/mol")
print(f"G_excess_mix range: {G_excess_mix.min():.2f} to {G_excess_mix.max():.2f} J/mol")

# Plot excess Gibbs energy
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(x_1_sorted, G_excess_mix, 'o-', linewidth=2.5, markersize=8, 
        label='G^E_mix', color='#d62728')

ax.set_xlabel('Mole fraction of Amine (x_1)', fontsize=14)
ax.set_ylabel('Excess Gibbs Energy (J/mol)', fontsize=14)
ax.set_title('Excess Gibbs Energy of Mixing\nAmine + H2O at T=313 K', 
             fontsize=15, fontweight='bold')

ax.set_xscale('log')
ax.grid(True, alpha=0.3, which='both')
ax.legend(fontsize=13)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Extract latent vectors for all compositions
def extract_latent_vectors(group_df, test_std, device='cuda'):
    """
    Extract latent vectors for one composition at one pco2 point.
    Returns: latent_vectors (tensor), component_names, mass_fractions
    """
    # Use first data point from the group
    idx = group_df.iloc[0]['index']
    base_data = test_std[idx]
    
    # Create DataLoader
    loader = DataLoader(
        dataset=[base_data],
        batch_size=1,
        shuffle=False,
        follow_batch=['component_batch']
    )
    
    batched_data = next(iter(loader)).to(device)
    
    model.eval()
    with torch.no_grad():
        _, latent_vectors, comp_emb = model(batched_data)
    
    return latent_vectors, batched_data.component_names, batched_data.component_mass_frac

print("Function ready. What plot format do you want?")

In [ ]:
# Step 1: Filter for T=298.15 K and extract data closest to pco2=15 kPa
target_T = 298.15
target_pco2 = 15.0

# Check if this temperature exists in system 62
system_62_data = df_test[df_test['system_id'] == 62]
available_temps = system_62_data['T'].unique()
print(f"Available temperatures for System 62: {sorted(available_temps)}")

# Find closest temperature if 298.15 doesn't exist
if target_T not in available_temps:
    closest_T = min(available_temps, key=lambda x: abs(x - target_T))
    print(f"T={target_T} K not found. Using closest: T={closest_T} K")
    target_T = closest_T

# Filter data
system_data = df_test[(df_test['system_id'] == 62) & (df_test['T'] == target_T)]

if len(system_data) == 0:
    print("No data found! Let's check what systems have data at 298.15 K:")
    temp_check = df_test[df_test['T'] == target_T]
    print(temp_check.groupby('system_id')['component_names'].first())
else:
    print(f"\nFound {len(system_data)} points at T={target_T} K")
    print(f"Available compositions: {system_data['mole_fractions'].nunique()}")

In [ ]:
# Step 2: Extract latent vectors for each composition at pco2 ≈ 15 kPa
target_T = 313.15
target_pco2 = 15.0

system_data = df_test[(df_test['system_id'] == 62) & (df_test['T'] == target_T)]
comp_groups = system_data.groupby('mole_fractions')

results = []

for mole_frac, group in comp_groups:
    # Find point closest to target pco2
    group_sorted = group.iloc[(group['pco2'] - target_pco2).abs().argsort()]
    closest_row = group_sorted.iloc[0]
    
    idx = closest_row['index']
    base_data = test_std[idx]
    
    # Extract latent vectors
    loader = DataLoader(
        dataset=[base_data],
        batch_size=1,
        shuffle=False,
        follow_batch=['component_batch']
    )
    batched_data = next(iter(loader)).to('cuda')
    
    model.eval()
    with torch.no_grad():
        _, latent_vectors, _ = model(batched_data)
    
    # Calculate gamma_i (mean across latent dimensions)
    gamma_i = latent_vectors.mean(dim=1).cpu().numpy()
    
    results.append({
        'mole_fractions': mole_frac,
        'gamma_i': gamma_i,
        'pco2_actual': closest_row['pco2'],
        'component_names': base_data.component_names
    })

print(f"Extracted latent vectors for {len(results)} compositions")
print(f"Component order: {results[0]['component_names']}")

In [ ]:
# Step 3: Plot activity coefficients vs mole fraction
fig, ax = plt.subplots(figsize=(10, 8))

# Extract data for plotting
x_pz = [r['mole_fractions'][0] for r in results]  # PZ mole fraction
x_tea = [r['mole_fractions'][1] for r in results]  # TEA mole fraction
x_h2o = [r['mole_fractions'][2] for r in results]  # H2O mole fraction

gamma_pz = [r['gamma_i'][0] for r in results]
gamma_tea = [r['gamma_i'][1] for r in results]
gamma_h2o = [r['gamma_i'][2] for r in results]

# Sort by PZ mole fraction for smooth lines
sorted_indices = np.argsort(x_pz)
x_pz_sorted = np.array(x_pz)[sorted_indices]
gamma_pz_sorted = np.array(gamma_pz)[sorted_indices]
gamma_tea_sorted = np.array(gamma_tea)[sorted_indices]
gamma_h2o_sorted = np.array(gamma_h2o)[sorted_indices]

# Plot
ax.plot(x_pz_sorted, gamma_pz_sorted, 'o-', linewidth=2, markersize=8, label='γ_PZ', color='#1f77b4')
ax.plot(x_pz_sorted, gamma_tea_sorted, 's-', linewidth=2, markersize=8, label='γ_TEA', color='#ff7f0e')
ax.plot(x_pz_sorted, gamma_h2o_sorted, '^-', linewidth=2, markersize=8, label='γ_H2O', color='#2ca02c')

ax.set_xlabel('Mole fraction of PZ (x_PZ)', fontsize=13)
ax.set_ylabel('Activity coefficient (γᵢ)', fontsize=13)
ax.set_title(f'Activity Coefficients vs Composition\nSystem 62: PZ + TEA + H2O at T={target_T} K', 
             fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, alpha=0.3, which='both')
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()

print("Activity coefficient plot complete!")

In [ ]:
def analyze_batch(model, batched_data, device='cpu'):
    """
    Perform forward pass and analyze a single batch.
    
    Args:
        model: Trained GDMPNN model
        batched_data: PyG Data object
        device: Device to run on ('cpu' or 'cuda')
    
    Returns:
        dict: Analysis results including predictions and intermediate values (all as tensors)
    """
    model.eval()
    batched_data = batched_data.to(device)
    
    with torch.no_grad():
        y_pred, latent_vectors, comp_emb = model(batched_data)
    
    # Unstandardize values
    T_actual = batched_data.T * batched_data.T_std + batched_data.T_mean
    pco2_actual = batched_data.pco2 * batched_data.pco2_std + batched_data.pco2_mean
    
    # Calculate thermodynamic quantities
    R = 8.314  # Gas constant J/(mol·K)
    gamma_i = latent_vectors.mean(dim=1).view(1, -1)
    G_excess = gamma_i * R * T_actual.view(-1, 1)
    G_excess_sum = G_excess.sum(dim=1)
    comp_emb_mean = comp_emb.mean(dim=1).view(1, -1)
    
    torch.set_printoptions(precision=2, sci_mode=False)
    # Print formatted output
    print("\n" + "="*120)
    print("FORWARD PASS INPUTS:")
    print("="*120)
    print(f"Component names                     : {batched_data.component_names}")
    print(f"Component mass fractions            : {batched_data.component_mass_frac}")
    print(f"Component mole fractions            : {batched_data.component_mole_frac}")
    print(f"System temperature                  : {T_actual} Kelvin")
    print(f"Partial pressure of CO2             : {pco2_actual} kPa")
    
    print("\n" + "="*120)
    print("FORWARD PASS OUTPUTS:")
    print("="*120)
    print(f"Predicted CO2 solubility            : {y_pred.detach()}")
    print(f"Actual CO2 solubility               : {batched_data.aco2}")
    print(f"Absolute error                      : {torch.abs(y_pred - batched_data.aco2).detach()}")
    print(f"Relative error (%)                  : {(torch.abs(y_pred - batched_data.aco2) / batched_data.aco2 * 100).detach()}")
    
    print("\n" + "="*120)
    print("SOLVENT SUBSYSTEM ANALYSIS:")
    print("="*120)
    print(f"Aggregated latent vector (γᵢ)       : {gamma_i.detach()}")
    print(f"Excess Gibbs Energy (Gᵢ_excess)     : {G_excess.detach()} J/mol")
    print(f"Sum of Excess Gibbs Energy          : {G_excess_sum.detach()} J/mol")
    print(f"Aggregated component embedding      : {comp_emb_mean.detach()}")
    print("="*120 + "\n")
    
    # Return results as dict (all tensors)
    results = {
        'component_names': batched_data.component_names,
        'mass_fractions': batched_data.component_mass_frac,
        'mole_fractions': batched_data.component_mole_frac,
        'temperature': T_actual,
        'pco2': pco2_actual,
        'predicted_aco2': y_pred.detach(),
        'actual_aco2': batched_data.aco2,
        'absolute_error': torch.abs(y_pred - batched_data.aco2).detach(),
        'relative_error': (torch.abs(y_pred - batched_data.aco2) / batched_data.aco2 * 100).detach(),
        'latent_vectors': latent_vectors.detach(),
        'gamma_i': gamma_i.detach(),
        'G_excess': G_excess.detach(),
        'G_excess_sum': G_excess_sum.detach(),
        'component_embeddings': comp_emb.detach(),
        'comp_emb_mean': comp_emb_mean.detach()
    }
    
    return results


def analyze_sample(model, dataset, sample_idx, device='cpu'):
    """
    Analyze a specific sample from the dataset.
    
    Args:
        model: Trained GDMPNN model
        dataset: PyG dataset
        sample_idx: Index of the sample to analyze
        device: Device to run on ('cpu' or 'cuda')
    
    Returns:
        dict: Analysis results (all tensors)
    """
    loader = DataLoader(
        dataset=dataset,
        batch_size=1,
        shuffle=False,
        follow_batch=['component_batch']
    )
    
    for i, batched_data in enumerate(loader):
        if i == sample_idx:
            return analyze_batch(model, batched_data, device)
    
    raise IndexError(f"Sample index {sample_idx} not found in dataset of size {len(dataset)}")

In [ ]:
results = analyze_sample(model, test_std, sample_idx=45, device='cuda')

## EQUILIBRIUM CURVE 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy

# Step 1: Get the base system
base_data = test_std[0]

# Unstandardize the original values to see what we're working with
T_unstd = base_data.T * base_data.T_std + base_data.T_mean
pco2_unstd = base_data.pco2 * base_data.pco2_std + base_data.pco2_mean

print(f"System ID: {base_data.system_id}")
print(f"System type: {base_data.system_type}")
print(f"Components: {base_data.component_names}")
print(f"Mole fractions: {base_data.component_mole_frac}")
print(f"Temperature: {T_unstd:.2f} K")
print(f"Original pco2: {pco2_unstd:.2f} kPa")
print(f"Original aco2: {base_data.aco2:.4f}")

# Step 2: Create pco2 sweep (1 to 1000 kPa, 100 points, log scale)
pco2_range_unstd = np.logspace(np.log10(1), np.log10(1000), 100)

# Step 3: Standardize the pco2 values (convert tensors to float)
pco2_mean = base_data.pco2_mean.item() if torch.is_tensor(base_data.pco2_mean) else base_data.pco2_mean
pco2_std = base_data.pco2_std.item() if torch.is_tensor(base_data.pco2_std) else base_data.pco2_std

pco2_range_std = (pco2_range_unstd - pco2_mean) / pco2_std

# Step 4: Create copies of the data with varying pco2
data_list = []
for pco2_std_val in pco2_range_std:
    data_copy = base_data.clone()
    data_copy.pco2 = torch.tensor(pco2_std_val, dtype=torch.float32)
    data_list.append(data_copy)

print(f"Created {len(data_list)} data points for inference")
print(f"pco2 range (unstd): {pco2_range_unstd[0]:.2f} to {pco2_range_unstd[-1]:.2f} kPa")
# Step 5: Run inference directly on the list
model.eval()
predictions = []
pco2_actual_list = []

with torch.no_grad():
    for data in data_list:
        # Create a mini-batch using DataLoader to handle component_batch_batch
        temp_loader = DataLoader(
            dataset=[data],
            batch_size=1,
            shuffle=False,
            follow_batch=['component_batch']
        )
        
        batched_data = next(iter(temp_loader))
        batched_data = batched_data.to('cuda')
        
        y_pred, _, _ = model(batched_data)
        predictions.append(y_pred.cpu().item())
        
        # Store actual pco2 for verification
        pco2_actual = (batched_data.pco2 * batched_data.pco2_std + batched_data.pco2_mean).cpu().item()
        pco2_actual_list.append(pco2_actual)

predictions = np.array(predictions)
pco2_actual_list = np.array(pco2_actual_list)

print(f"Inference complete!")
print(f"Prediction range: {predictions.min():.4f} to {predictions.max():.4f}")
print(f"pco2 check: {pco2_actual_list[0]:.2f} to {pco2_actual_list[-1]:.2f} kPa")
# Step 6: Plot the equilibrium curve
plt.figure(figsize=(10, 6))
plt.plot(pco2_actual_list, predictions, 'b-', linewidth=2, label='Model prediction')
plt.scatter([pco2_unstd], [base_data.aco2], color='red', s=100, zorder=5, 
            label=f'Original data point (pCO₂={pco2_unstd:.2f} kPa)')

plt.xlabel('pCO₂ (kPa)', fontsize=12)
plt.ylabel('αCO₂ (mol CO₂/mol amine)', fontsize=12)
plt.title(f'CO₂ Equilibrium Curve\n{base_data.system_type}: {base_data.component_names}\nT = {T_unstd:.2f} K', 
          fontsize=13)
plt.xscale('log')
plt.grid(True, alpha=0.3, which='both')
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

print(f"\nSystem info:")
print(f"  Components: {base_data.component_names}")
print(f"  Mole fractions: {base_data.component_mole_frac.cpu().numpy()}")
print(f"  Temperature: {T_unstd:.2f} K")
print(f"  pCO₂ range: {pco2_actual_list[0]:.2f} - {pco2_actual_list[-1]:.2f} kPa")
print(f"  αCO₂ range: {predictions.min():.4f} - {predictions.max():.4f}")

In [ ]:
# Step 1: Analyze test set to find unique equilibrium systems
import pandas as pd

# Extract key info from each test sample
test_info = []
for i, data in enumerate(test_std):
    T_unstd = (data.T * data.T_std + data.T_mean).item()
    pco2_unstd = (data.pco2 * data.pco2_std + data.pco2_mean).item()
    
    # Convert mole fractions to tuple for grouping
    mole_frac_tuple = tuple(data.component_mole_frac.cpu().numpy().round(4))
    
    test_info.append({
        'index': i,
        'system_id': data.system_id,
        'component_names': str(data.component_names),
        'mole_fractions': mole_frac_tuple,
        'T': round(T_unstd, 2),
        'pco2': pco2_unstd,
        'aco2': data.aco2,
        'system_type': data.system_type
    })

df_test = pd.DataFrame(test_info)

# Group by system characteristics
grouped = df_test.groupby(['system_id', 'component_names', 'mole_fractions', 'T'])

print(f"Total test samples: {len(test_std)}")
print(f"Number of unique equilibrium curves: {len(grouped)}")
print(f"\nFirst 10 unique systems:")
print(grouped.size().head(10))

In [ ]:
# Check the breakdown
print(f"Unique systems (by system_id): {df_test['system_id'].nunique()}")
print(f"Unique equilibrium curves (system + T + composition): {len(grouped)}")
print(f"\nBreakdown by system_id:")
system_breakdown = df_test.groupby('system_id').agg({
    'T': lambda x: len(x.unique()),  # number of unique temperatures
    'pco2': 'count'  # total data points
}).rename(columns={'T': 'num_temperatures', 'pco2': 'num_datapoints'})
print(system_breakdown.head(15))

In [ ]:
# Option 1: Filter curves with at least N data points
min_points = 5
filtered_groups = grouped.filter(lambda x: len(x) >= min_points).groupby(['system_id', 'component_names', 'mole_fractions', 'T'])
print(f"Curves with >= {min_points} points: {len(filtered_groups)}")

# Option 2: Show distribution of points per curve
points_per_curve = grouped.size()
print(f"\nDistribution of data points per curve:")
print(points_per_curve.describe())
print(f"\nCurves with 1 point: {(points_per_curve == 1).sum()}")
print(f"Curves with 2-4 points: {((points_per_curve >= 2) & (points_per_curve <= 4)).sum()}")
print(f"Curves with 5+ points: {(points_per_curve >= 5).sum()}")
print(f"Curves with 10+ points: {(points_per_curve >= 10).sum()}")

In [ ]:
# Step 2: Select 6 random curves with >= 5 points
min_points = 5
filtered_groups = grouped.filter(lambda x: len(x) >= min_points).groupby(['system_id', 'component_names', 'mole_fractions', 'T'])

# Sample 6 random curves
import random
random.seed(42)
sampled_keys = random.sample(list(filtered_groups.groups.keys()), 6)

print(f"Selected 6 curves:")
for i, key in enumerate(sampled_keys):
    system_id, comp_names, mole_frac, T = key
    num_points = len(filtered_groups.get_group(key))
    print(f"{i+1}. System {system_id}: {comp_names}, T={T} K, {num_points} points")

In [ ]:
# Step 3: Function to generate equilibrium curve for a specific group
def generate_equilibrium_curve(group_df, test_std, pco2_range=(1, 1000), num_points=100):
    """
    Generate model predictions for an equilibrium curve.
    
    Args:
        group_df: DataFrame slice for one equilibrium curve
        test_std: Test dataset
        pco2_range: (min, max) pco2 in kPa
        num_points: Number of points to generate
    
    Returns:
        pco2_sweep: Array of pco2 values (unstandardized)
        aco2_pred: Array of predicted aco2 values
        pco2_exp: Experimental pco2 values
        aco2_exp: Experimental aco2 values
    """
    # Get base data from first point in group
    base_idx = group_df.iloc[0]['index']
    base_data = test_std[base_idx]
    
    # Extract experimental points
    pco2_exp = group_df['pco2'].values
    aco2_exp = group_df['aco2'].values
    
    # Generate pco2 sweep
    pco2_sweep_unstd = np.logspace(np.log10(pco2_range[0]), np.log10(pco2_range[1]), num_points)
    
    # Standardize
    pco2_mean = base_data.pco2_mean.item()
    pco2_std = base_data.pco2_std.item()
    pco2_sweep_std = (pco2_sweep_unstd - pco2_mean) / pco2_std
    
    # Create data copies
    data_list = []
    for pco2_val in pco2_sweep_std:
        data_copy = base_data.clone()
        data_copy.pco2 = torch.tensor(pco2_val, dtype=torch.float32)
        data_list.append(data_copy)
    
    # Run inference
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in data_list:
            temp_loader = DataLoader(
                dataset=[data],
                batch_size=1,
                shuffle=False,
                follow_batch=['component_batch']
            )
            batched_data = next(iter(temp_loader))
            batched_data = batched_data.to('cuda')
            y_pred, _, _ = model(batched_data)
            predictions.append(y_pred.cpu().item())
    
    return pco2_sweep_unstd, np.array(predictions), pco2_exp, aco2_exp

print("Function defined. Ready to plot!")

In [ ]:
# Step 4: Plot the 6 example curves with formatted mass fractions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, key in enumerate(sampled_keys):
    system_id, comp_names, mole_frac, T = key
    group_df = filtered_groups.get_group(key)
    
    # Get base data to extract mass fractions
    base_idx = group_df.iloc[0]['index']
    base_data = test_std[base_idx]
    
    # Parse and format mass fractions
    mass_fracs = [float(x) for x in base_data.component_mass_frac.split(' + ')]
    mass_frac_str = ', '.join([f'{x:.2f}' for x in mass_fracs])
    
    # Format component names without brackets
    comp_names_str = ', '.join(eval(comp_names))  # Convert string list to actual list
    
    # Generate curve
    pco2_sweep, aco2_pred, pco2_exp, aco2_exp = generate_equilibrium_curve(group_df, test_std)
    
    # Plot
    ax = axes[i]
    ax.plot(pco2_sweep, aco2_pred, 'b-', linewidth=2, label='Model')
    ax.scatter(pco2_exp, aco2_exp, color='red', s=80, zorder=5, label='Experimental', edgecolors='black', linewidths=1)
    
    ax.set_xlabel('pCO₂ (kPa)', fontsize=10)
    ax.set_ylabel('αCO₂', fontsize=10)
    ax.set_title(f'System {system_id}: {comp_names_str}\nw = {mass_frac_str}, T = {T} K', fontsize=8)
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3, which='both')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("Plotting complete!")

In [ ]:
# Check which systems have multiple compositions at same temperature
composition_variety = df_test.groupby(['system_id', 'component_names', 'T']).agg({
    'mole_fractions': lambda x: len(set(x)),
    'index': 'count'
}).rename(columns={'mole_fractions': 'num_compositions', 'index': 'num_points'})

# Filter for systems with multiple compositions at same T
multi_comp = composition_variety[composition_variety['num_compositions'] > 1]
print(f"Systems with multiple compositions at same temperature:")
print(multi_comp.head(10))

In [ ]:
# Step 1: Extract system 62 at T=313.15 K with all compositions
target_system_id = 62
target_T = 313.15

# Filter the test dataframe
system_data = df_test[(df_test['system_id'] == target_system_id) & 
                       (df_test['T'] == target_T)]

# Group by composition
comp_groups = system_data.groupby('mole_fractions')

print(f"System {target_system_id} at T={target_T} K")
print(f"Number of different compositions: {len(comp_groups)}")
print(f"\nCompositions:")
for i, (mole_frac, group) in enumerate(comp_groups):
    # Get mass fractions from first sample
    idx = group.iloc[0]['index']
    mass_frac_str = test_std[idx].component_mass_frac
    print(f"{i+1}. x={mole_frac}, w={mass_frac_str}, {len(group)} points")